In [1]:
from py2neo import Graph, Node, Relationship

In [2]:
graph = Graph(password="password")

query = "match (n) return head(labels(n)) as label, count(*);"
result = graph.run(query)

print(result.to_data_frame())

   label  count(*)
0   Post     39929
1   User     66951
2    Tag       489
3  Stack      6187


#### Create constraints and indices - easier in browser - localhost:7474

In [3]:
# create index on :Post(title);
# create index on :Post(createdAt);
# create index on :Post(score);
# create index on :Post(views);
# create index on :Post(favorites);
# create index on :Post(answers);
# create index on :Post(score);

# create index on :User(name);
# create index on :User(createdAt);
# create index on :User(reputation);
# create index on :User(age);

# create index on :Tag(count);

# create constraint on (t:Tag) assert t.tagId is unique;
# create constraint on (u:User) assert u.userId is unique;
# create constraint on (p:Post) assert p.postId is unique;

# We then waited for the indexes to be finished.

# schema await

In [4]:
query = "match (u:User) with u,size( (u)-[:POSTED]->()) as posts order by posts desc limit 10 return u.displayname, posts;" 
result = graph.run(query)

print(result.to_data_frame())

    u.displayname  posts
0     Neil Slater    403
1     Vaalizaadeh    354
2         n1k31t4    347
3    Anony-Mousse    325
4  Brian Spiering    241
5            Emre    227
6    Martin Thoma    224
7         Dawny33    215
8           Tasos    205
9        JahKnows    203


#### Get number of tags used in asking questions

In [5]:
query = "match (u:User)-[:POSTED]->()-[:HAS_TAG]->(t:Tag) where u.displayname = 'Vaalizaadeh'  return t,count(*) as posts order by posts desc limit 5;" 
result = graph.run(query)

print(result.to_data_frame())

# This user only answered questions

                               t  posts
0  {'tagId': 'machine-learning'}     20
1     {'tagId': 'deep-learning'}     15
2    {'tagId': 'neural-network'}     11
3               {'tagId': 'cnn'}      4
4             {'tagId': 'keras'}      2


#### Get tags that User answered

In [6]:
query = "match (u:User)-[:POSTED]->()-[:HAS_TAG]->(t:Tag) where u.displayname = 'Puffin GDI' return t.tagId,count(*) as posts order by posts desc limit 5;" 
result = graph.run(query)

print(result.to_data_frame())

            t.tagId  posts
0           bigdata      1
1            libsvm      1
2  machine-learning      1


#### All shortest paths - run in neo4j console or browser

In [7]:
# MATCH path = allShortestPaths(
#      (u:User {displayname:"Emre"})-[*]-(me:User {displayname:"Vaalizaadeh"}))
# RETURN path;

![title](images/all_short_path.png)

#### User with letter t in name answered the most questions about python

In [45]:
query = "MATCH (u:User)-[:POSTED]->(answer)<-[:PARENT_OF]-()-[:HAS_TAG]-(:Tag {tagId:'python'}) WHERE u.displayname contains 't' RETURN u.displayname AS name,u.reputation AS rep,u.location, count(distinct answer) AS answers ORDER BY answers DESC LIMIT 20;" 
result = graph.run(query)
#print(result.to_data_frame())
xo = result.to_data_frame()
xo["rep/ans"] = xo["rep"].astype(float) / xo["answers"].astype(float)
#print(xo)
xo.head(40)

# query = "" 
# result = graph.run(query)
# print(result.to_data_frame())

,name,rep,u.location,answers,rep/ans
0,n1k31t4,8181,Europa,117,69.923077
1,Neil Slater,18824,"Durham, United Kingdom",36,522.888889
2,Juan Esteban de la Calle,1504,"Bogotá, Colombia",24,62.666667
3,Jan van der Vegt,6995,Amsterdam,23,304.130435
4,Aditya,1577,"West Bengal, India",22,71.681818
5,Stephen Rauch,1598,"Northern California, United States",19,84.105263
6,Peter,1648,"Frankfurt am Main, Deutschland",17,96.941176
7,Ankit Seth,1159,None,17,68.176471
8,Kiritee Gak,1379,"Delhi, India",15,91.933333
9,Shamit Verma,1754,"Mumbai, India",14,125.285714


In [130]:
query = "MATCH (u:User)-[:POSTED]->(answer)<-[:PARENT_OF]-()-[:HAS_TAG]-(:Tag {tagId:'python'}) WHERE u.displayname contains 't' RETURN u.displayname AS name,u.reputation AS rep,u.location, count(distinct answer) AS answers ORDER BY rep DESC;" 
result = graph.run(query)
#print(result.to_data_frame())
xr = result.to_data_frame()
xr["rep/ans"] = xr["rep"].astype(float) / xr["answers"].astype(float)
#print(xo)
#xr.head(40)
sort_by2 = xr.sort_values('answers', ascending=False)
#df.sort_values(by=['Brand'], inplace=True, ascending=False)
s2 = sort_by2.head(20)
s2.head(20)

,name,rep,u.location,answers,rep/ans
20,n1k31t4,8181,Europa,117,69.923077
230,Neil Slater,18824,"Durham, United Kingdom",36,522.888889
288,Juan Esteban de la Calle,1504,"Bogotá, Colombia",24,62.666667
34,Jan van der Vegt,6995,Amsterdam,23,304.130435
275,Aditya,1577,"West Bengal, India",22,71.681818
272,Stephen Rauch,1598,"Northern California, United States",19,84.105263
259,Peter,1648,"Frankfurt am Main, Deutschland",17,96.941176
352,Ankit Seth,1159,None,17,68.176471
304,Kiritee Gak,1379,"Delhi, India",15,91.933333
245,Shamit Verma,1754,"Mumbai, India",14,125.285714


In [65]:
sort_by = xo.sort_values('rep/ans', ascending=False)
#df.sort_values(by=['Brand'], inplace=True, ascending=False)
sort_by.head(20)

,name,rep,u.location,answers,rep/ans
1,Neil Slater,18824,"Durham, United Kingdom",36,522.888889
12,Franck Dernoncourt,3656,"Adobe Research, San Jose, CA, USA",10,365.600000
3,Jan van der Vegt,6995,Amsterdam,23,304.130435
17,Ryan Zotti,2564,"San Francisco, CA, USA",9,284.888889
19,StatsSorceress,1194,None,8,149.250000
14,stmax,1409,Austria,10,140.900000
13,Elias Strehle,1311,"Hamburg, Germany",10,131.100000
15,I_Play_With_Data,1294,USA,10,129.400000
9,Shamit Verma,1754,"Mumbai, India",14,125.285714
18,thanatoz,900,India,8,112.500000


In [143]:
import plotly_express as px         
px.bar(the3, x='name', y='rep/ans', template='plotly_dark')

In [139]:
fig = px.scatter(the3, x="answers", y="rep", color="name", size="rep/ans", symbol="rep/ans", log_x=True, log_y=True, template='plotly_dark')
fig.show()

In [140]:
fig2 = px.scatter(the3, x="rep", y="rep/ans", size="answers", symbol="answers", color="name", log_x=True, log_y=True, template='plotly_dark')
fig2.show()

In [141]:
px.box(data_frame=the3
    , y="rep/ans"
    , title="Distribution of Reputation/Answers"
    , template='plotly_dark'
    )

In [128]:
px.scatter_matrix(data_frame=sort_by
                  , dimensions=["rep", "answers"] 
                  , color="name"
                  , size="rep/ans"
                  , symbol="rep" 
                  , title='Comparisons'
                  , hover_name='name'
                  , template='seaborn'
                 )

In [47]:
# MATCH (u:User)-[:POSTED]->(answer)<-[:PARENT_OF]-()-[:HAS_TAG]-(:Tag {tagId:"nosql"}) 
# WHERE u.displayname = 'Emre'
# RETURN u.displayname, u.reputation,u.location, count(distinct answer) AS answers
# ORDER BY answers DESC;

#### Run this graph in browser/ console

In [62]:
# MATCH (u:User)-[:POSTED]->(answer)<-[:PARENT_OF]-(question)-[:HAS_TAG]-(tag:Tag {tagId:"python"}) 
# WHERE u.displayname contains "t"
# RETURN *,count(distinct answer) AS answers
# ORDER BY answers DESC limit 30;

![title](images/python_answers.png)

#### Top 5 tags of all time

In [73]:
query = "match (t:Tag) with t order by t.count desc limit 5 return t.tagId, t.count;" 
result = graph.run(query)
print(result.to_data_frame())

            t.tagId t.count
0  machine-learning    None
1           bigdata    None
2       data-mining    None
3         databases    None
4       definitions    None


#### Co-occurence with 'javascript' tag

In [75]:
query = "match (t:Tag {tagId:'javascript'})<-[:HAS_TAG]-()-[:HAS_TAG]->(other:Tag) WITH other, count(*) as freq order by freq desc limit 5 RETURN other.tagId,freq;" 
result = graph.run(query)
print(result.to_data_frame())

# match (t:Tag {tagId:"javascript"})<-[:HAS_TAG]-()-[:HAS_TAG]->(other:Tag) 
# WITH other, count(*) as freq order by freq desc limit 5
# RETURN other.tagId,freq;

        other.tagId  freq
0     visualization    11
1    neural-network     6
2  machine-learning     6
3            python     3
4            graphs     2


#### Most active answerers for python

In [79]:
query = "match (t:Tag {tagId:'python'})<-[:HAS_TAG]-()-[:PARENT_OF]->()<-[:POSTED]-(u:User) WITH u, count(*) as freq order by freq desc limit 10 RETURN u.displayname,freq;" 
result = graph.run(query)
print(result.to_data_frame())


# match (t:Tag {tagId:"neo4j"})<-[:HAS_TAG]-()
#        -[:PARENT_OF]->()<-[:POSTED]-(u:User) 
# WITH u, count(*) as freq order by freq desc limit 10
# RETURN u.name,freq;

    u.displayname  freq
0         n1k31t4   117
1        JahKnows    52
2  Brian Spiering    49
3    Anony-Mousse    49
4            Emre    42
5     Vaalizaadeh    38
6     Neil Slater    36
7            MaxU    34
8  Kasra Manshaei    31
9           Tasos    28


#### Where are the most active answerers also active

In [82]:
query = "MATCH (neo:Tag {tagId:'python'})<-[:HAS_TAG]-()-[:PARENT_OF]->()<-[:POSTED]-(u:User) WITH neo,u, count(*) as freq order by freq desc limit 10 MATCH (u)-[:POSTED]->()<-[:PARENT_OF]-(p)-[:HAS_TAG]->(other:Tag) WHERE NOT (p)-[:HAS_TAG]->(neo) WITH u,other,count(*) as freq2 order by freq2 desc RETURN u.displayname,collect(distinct other.tagId)[1..5] as tags;" 
result = graph.run(query)
print(result.to_data_frame())


# MATCH (neo:Tag {tagId:'neo4j'})<-[:HAS_TAG]-()
#       -[:PARENT_OF]->()<-[:POSTED]-(u:User) 
# WITH neo,u, count(*) as freq order by freq desc limit 10
# MATCH (u)-[:POSTED]->()<-[:PARENT_OF]-(p)-[:HAS_TAG]->(other:Tag)
# WHERE NOT (p)-[:HAS_TAG]->(neo)
# WITH u,other,count(*) as freq2 order by freq2 desc 
# RETURN u.displayname,collect(distinct other.tagId)[1..5] as tags;

    u.displayname                                               tags
0     Vaalizaadeh        [deep-learning, neural-network, cnn, keras]
1    Anony-Mousse  [machine-learning, k-means, data-mining, class...
2     Neil Slater  [neural-network, reinforcement-learning, deep-...
3            Emre  [neural-network, classification, deep-learning...
4         n1k31t4  [deep-learning, neural-network, keras, tensorf...
5        JahKnows  [neural-network, deep-learning, keras, classif...
6  Brian Spiering  [nlp, deep-learning, neural-network, text-mining]
7  Kasra Manshaei  [clustering, graphs, classification, data-mining]
8           Tasos             [pandas, nlp, statistics, time-series]


#### Posted most questions about neo4j

In [85]:
query = "MATCH (t:Tag {tagId:'neo4j'})<-[:HAS_TAG]-(:Post)<-[:POSTED]-(u:User) RETURN u.displayname,count(*) as count ORDER BY count DESC LIMIT 10;" 
result = graph.run(query)
print(result.to_data_frame())

# MATCH (t:Tag {tagId:'neo4j'})<-[:HAS_TAG]-(:Post)<-[:POSTED]-(u:User)
# RETURN u.name,count(*) as count
# ORDER BY count DESC LIMIT 10;

       u.displayname  count
0        John Newman      2
1       Sreejithc321      2
2        TobiasJakob      1
3           blunders      1
4                SRS      1
5      sheldonkreger      1
6             Victor      1
7  Austin Capobianco      1


#### Top answerers for pandas tag

In [88]:
query = "MATCH (:Tag {tagId:'pandas'})<-[:HAS_TAG]-()-[:PARENT_OF]->()<-[:POSTED]-(u:User) RETURN u.displayname,count(*) as count ORDER BY count DESC LIMIT 10;" 
result = graph.run(query)
print(result.to_data_frame())

# MATCH (:Tag {tagId:'py2neo'})<-[:HAS_TAG]-()-[:PARENT_OF]->()
#       <-[:POSTED]-(u:User)
# RETURN u.displayname,count(*) as count
# ORDER BY count DESC LIMIT 10;

   u.displayname  count
0        n1k31t4     36
1           MaxU     24
2           Emre     18
3          Tasos     15
4         Aditya     12
5          bkshi     12
6  Stephen Rauch     10
7    Vaalizaadeh     10
8   TwinPenguins      8
9       JahKnows      8


#### Users who answered their own question 

In [94]:
query = "MATCH (u:User) WHERE u.reputation > 1000 MATCH (u)-[:POSTED]->(question)-[:ANSWER]->(answer)<-[:POSTED]-(u) WITH u,count(distinct question) AS questions ORDER BY questions DESC LIMIT 5 RETURN u.displayname, u.reputation, questions;" 
result = graph.run(query)
print(result.to_data_frame())

# MATCH (u:User) WHERE u.reputation > 20000
# MATCH (u)-[:POSTED]->(question)-[:ANSWER]->(answer)<-[:POSTED]-(u)
# WITH u,count(distinct question) AS questions
# ORDER BY questions DESC LIMIT 5
# RETURN u.displayname, u.reputation, questions;

# not working no :ANSWER relationship

Empty DataFrame
Columns: []
Index: []


#### Smaller queries

In [ ]:
# MATCH path = allShortestPaths( (t:Tag {tagId:"activation"})-[*]-(u:Tag {tagId:"xboost"})) RETURN path LIMIT 10;

![title](images/xboost_activation_bigdb.png)

In [1]:
# MATCH path = allShortestPaths( (t:User {displayname:"black_cat"})-[*]-(u:User {displayname:"Skier"})) RETURN path LIMIT 10;

![title](images/black_cat_to_skier.png)

In [138]:
query = "MATCH (u:User)-[:POSTED]->(answer)<-[:PARENT_OF]-()-[:HAS_TAG]-(:Tag) RETURN u.displayname AS name,u.reputation AS rep,u.location, count(distinct answer) AS answers ORDER BY answers DESC LIMIT 20;" 
result = graph.run(query)
#print(result.to_data_frame())
the3 = result.to_data_frame()
the3["rep/ans"] = the3["rep"].astype(float) / the3["answers"].astype(float)
#print(xo)
the3.head(40)



,name,rep,u.location,answers,rep/ans
0,Neil Slater,18824,"Durham, United Kingdom",393,47.898219
1,n1k31t4,8181,Europa,345,23.713043
2,Vaalizaadeh,8710,None,329,26.474164
3,Anony-Mousse,5971,Series of tubes.,325,18.372308
4,Brian Spiering,5197,"San Francisco, CA, USA",238,21.836134
5,Emre,8881,"Silicon Valley, CA, United States",227,39.123348
6,JahKnows,5782,"Montreal, QC, Canada",203,28.482759
7,Jan van der Vegt,6995,Amsterdam,184,38.016304
8,Esmailian,5069,None,152,33.348684
9,Kasra Manshaei,3900,"Bonn, Germany",147,26.530612


In [142]:
sort = the3.sort_values('rep/ans', ascending=False)
#df.sort_values(by=['Brand'], inplace=True, ascending=False)
sort.head(20)

,name,rep,u.location,answers,rep/ans
16,Martin Thoma,7157,"München, Deutschland",108,66.268519
14,Dawny33,5798,"Gurgaon, India",112,51.767857
0,Neil Slater,18824,"Durham, United Kingdom",393,47.898219
19,ncasas,4201,"Barcelona, España",103,40.786408
5,Emre,8881,"Silicon Valley, CA, United States",227,39.123348
7,Jan van der Vegt,6995,Amsterdam,184,38.016304
11,oW_,3906,"San Diego, CA, United States",116,33.672414
8,Esmailian,5069,None,152,33.348684
6,JahKnows,5782,"Montreal, QC, Canada",203,28.482759
9,Kasra Manshaei,3900,"Bonn, Germany",147,26.530612


In [179]:
query = "MATCH (n:Post) with n order by n.comments DESC RETURN n.score as score, n.comments as comments, n.postId as postId, n.body as body LIMIT 160;" 
# match (t:Tag) with t order by t.count desc limit 5 return t.tagId, t.count;
hyp4 = graph.run(query)
#print(result.to_data_frame())
hyp5 = hyp4.to_data_frame()
hyp5["score/comments"] = hyp5["score"].astype(float) / hyp5["comments"].astype(float)
hyp5.head(40)

,score,comments,postId,body,score/comments
0,4,9,43471,<p>I have a data set of total 8000 sound sampl...,0.444444
1,120,9,24,<p>The standard k-means algorithm isn't direct...,13.333333
2,4,9,34116,<p>It sounds like dump question but as a begin...,0.444444
3,0,9,43351,<p>It's a little unclear as to what you gain b...,0.000000
4,1,9,46350,<p>I am working on a LSTM network that I get l...,0.111111
5,1,9,10054,<p>I've done something like you're describing ...,0.111111
6,3,9,24741,<h3>Answer</h3><blockquote> <p>Are there any ...,0.333333
7,1,9,34016,<p><br>I wanted to start off by saying this is...,0.111111
8,2,9,38908,<p>Using max-pooling is not a good idea on its...,0.222222
9,0,9,43305,<p>i m try to apply k-means with Python 3 to m...,0.000000


In [180]:
import plotly_express as px         
px.bar(hyp5, x='body', y='score', template='plotly_dark')

In [181]:
px.box(data_frame=hyp5
    , y="score/comments"
    , title="Distribution of Post Score in Posts with 9 Answers"
    , template='plotly_dark'
    )

In [183]:
# MATCH (n:Post) where n.postId="155" RETURN n.body

query = "MATCH (n:Post) where n.postId='155' RETURN n.body;" 
# match (t:Tag) with t order by t.count desc limit 5 return t.tagId, t.count;
hyp6 = graph.run(query)
#print(result.to_data_frame())
hyp7 = hyp6.to_data_frame()
#hyp5["score/comments"] = hyp5["score"].astype(float) / hyp5["comments"].astype(float)
hyp7.head(40)

,n.body
0,<p>One of the common problems in data science ...


In [191]:
#display(hyp7)

print(hyp7.to_string())

                                              n.body
0  <p>One of the common problems in data science ...
